# Calibrate distance Source-Detector at VULCAN

We will be using Al Bragg edge peaks for this on the Open Beam (OB) data

In [ ]:
from PIL import Image
import numpy as np

In [ ]:
from pprint import pprint
from ipywidgets import widgets
from ipywidgets.widgets import interact
from IPython.display import display

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

In [ ]:
import glob

In [ ]:
#%matplotlib notebook
%matplotlib inline

In [ ]:
import pyfits

## Setup plot.ly 

In [ ]:
from plotly.offline  import plot
from plotly.graph_objs import *
import plotly.tools as lts
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.plotly as py

In [ ]:
init_notebook_mode()

## Load OB data

**USER** =====>  which OB folder to use

In [ ]:
# root dir
root_dir = '/Volumes/My Book Thunderbolt Duo/IPTS/BraggEdge/VULCAN/Runs#39_3600sec_#40_601sec_Open_Beam/'

In [ ]:
# OBs
data_dir = root_dir + 'data_image040'
print(data_dir)

In [ ]:
list_fname = glob.glob(data_dir + '/*')

In [ ]:
pprint("Number of files to load: %d" %(len(list_fname)))

Make sure files are sorted the right way !

In [ ]:
#pprint(list_fname)

Load images

In [ ]:
nbr_data = len(list_fname)
OBs = []

w = widgets.IntProgress()
w.max = nbr_data
display(w)

for _index in range(nbr_data):
        
        _file = list_fname[_index]
        hdu_list = pyfits.open(_file)  # fits
        hdu = hdu_list[0]
        _image = hdu.data
        #_image = Image.open(_file)  # tif
        
        _image = np.asarray(_image)
        OBs.append(_image)

        w.value = _index
    

In [ ]:
#test display image
fig2 = plt.figure()
plt.imshow(OBs[200])

## Add all counts of each image

In [ ]:
nbr_obs = len(OBs)

In [ ]:
w1 = widgets.IntProgress()
w1.max = nbr_data
display(w1)

sum_counts = []
for _index, _image in enumerate(OBs):
    _sum = np.sum(_image)
    sum_counts.append(_sum)
    w1.value = _index

In [ ]:
# display to check values
trace1 = go.Scatter(
            x = np.arange(nbr_obs),
            y = sum_counts)
data = [trace1]

#layout = dict(title = 'Total Counts',
#        xaxis = dict(title = 'Image index'),
#        yaxis = dict(title = 'Counts')
#              )
# or
#
layout = go.Layout(
        title = "Total Counts",
        xaxis = {"title": 'Image index'},
        yaxis = {"title": "Counts"}
    )

figure = go.Figure(data = data, layout = layout)
iplot(figure)

# Convert 'Image Index' to 'time' 

In [ ]:
import sys
sys.path.append('/users/j35/git/braggedge/python')

In [ ]:
from neutronbraggedge.experiment_handler import *

In [ ]:
import os
tof_spectra_file = os.path.join(root_dir, 'Image040_Spectra.txt')
_tof_handler = TOF(filename = tof_spectra_file)
print("TOF axis size: %d" %len(_tof_handler.tof_array))
print("Number of files: %d" %nbr_obs)

In [ ]:
layout = go.Layout(title = "Sum Counts vs TOF")
layout.xaxis['title'] = "TOF (microS)"
layout.yaxis['title'] = "Counts"
trace1 = Scatter(x = _tof_handler.tof_array,
                y = sum_counts,
                mode = 'markers',
                marker = Marker( color = 'red',
                               symbol = 'square'))
data = Data([trace1])
fig = Figure(data=data, layout=layout)
iplot(fig)

# Retrieve Al Bragg edges values 

In [ ]:
from neutronbraggedge.braggedge import BraggEdge

In [ ]:
_handler = BraggEdge(material=['Al', 'Fe'], number_of_bragg_edges=4)

In [ ]:
bragg_edges = _handler.bragg_edges
hkl = _handler.hkl

In [ ]:
print(len(bragg_edges))

# Estimate best distance_source_detector 

In [ ]:
detector_offset = 4.5e3 #micros   #default 4.5e3

In [ ]:
print(hkl)

In [ ]:
print(bragg_edges)

In [ ]:
distance_source_detector = 24.28  #m

# initialize tof -> lambda convertor
_exp = Experiment(tof = _tof_handler.tof_array,
                 distance_source_detector_m = distance_source_detector,
                 detector_offset_micros = detector_offset)

# retrieve lambda array
lambda_array_angstroms = _exp.lambda_array * 1e10

# format hkl labels
_hkl_formated = {}
for _material in hkl:
    _hkl_string = []
    for _hkl in hkl[_material]:
        _hkl_s = ",".join(str(x) for x in _hkl)
        _hkl_s = _material + "\n" + _hkl_s
        _hkl_string.append(_hkl_s)
    _hkl_formated[_material] = _hkl_string

# plot
trace1 = go.Scatter(
            x = lambda_array_angstroms,
            y = sum_counts,
            mode = "markers")

layout = go.Layout(
            title = "Total Counts and Ideal Al Bragg Edge Values",
            xaxis = {"title": "Lambda (Angstroms)"},
            yaxis = {"title": "Counts"},
            annotations = []
    )

max_x = 6
y_off = 1

for y_index, _material in enumerate(bragg_edges):
    for _index, _value in enumerate(bragg_edges[_material]):
        if _value > max_x:
            continue
        bragg_line = {"type": "line",
                    'x0': _value,
                    'x1': _value,
                     'yref': "paper",
                     'y0': 0,
                     'y1': 1,
                     'line': {
                        'color': 'rgb(255, 0, 0)',
                        'width': 1
            }}
        layout.shapes.append(bragg_line)
        y_off = 1 - 0.25 * y_index
    
        # add labels to plots
        _annot = dict(
                    x=_value,
                    y= y_off,
                    text = _hkl_formated[_material][_index],
                    yref="paper",
                    font=dict(
                        family="Arial",
                        size=16,
                        color="rgb(150,50,50)"
                    ),
                    showarrow=True,
                    arrowhead=3,
                    ax=0,
                    ay=-25)
                
        layout.annotations.append(_annot)
    
    

data = [trace1]
    
fig1 = go.Figure(data=data, layout=layout)
iplot(fig1)    

END OF CURRENT WORK